In [1]:
from flask import Flask,render_template,url_for,request

import pandas as pd 
import numpy as np 
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.metrics import accuracy_score,classification_report
from sklearn.feature_extraction.text import TfidfVectorizer , CountVectorizer

In [2]:
from flask import Flask,render_template,request,jsonify

import pandas as pd 
import numpy as np 
import re
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

app = Flask(__name__)

@app.route('/')

def home():
    return render_template('home.html')

@app.route('/predict',methods=['POST'])

def remove_unwant(x):
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
 
    for ele in x:
        if ele in punc:
            x = x.replace(ele, "")
            
    return x 

def predict():
    df= pd.read_csv("all_kindle_review .csv")
    df = df.dropna()
    df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
    
    df.loc[(df['rating']==2)|(df['rating']==1),'rating']= -1
    df.loc[df['rating']==3,'rating']= 0
    df.loc[(df['rating']==4)|(df['rating']==5),'rating']= 1

    Sentiment_words=[]

    for row in df['rating']:
        if row == -1:
            Sentiment_words.append('negative')
        elif row == 0:
            Sentiment_words.append('neutral')    
        elif row == 1:
            Sentiment_words.append('positive')
    
    
    df['Sentiment_words'] = Sentiment_words
    # Features and Labels
    
    df['summary'] = df['summary'].apply(lambda x:remove_unwant(x))
    df['summary'] = df['summary'].apply(lambda x:x.lower())
    
    X = df['summary']
    y = df['rating']
    
    # Extract Feature With tfidfVectorizer
    
    tf=TfidfVectorizer(ngram_range=(1,1), use_idf=1)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    #Naive Bayes Classifier
    X_train = tf.fit_transform(X_train)
    X_test = tf.transform(X_test)
    
    clf = MultinomialNB()
    
    clf.fit(X_train,y_train)
    
    clf.score(X_test,y_test)


    if request.method == 'POST':
        
        message = request.form['message']
        vect = tf.transform(message)
        my_pred = clf.predict(vect)
        
        if my_pred == 1:
            pred = 'positive'
        elif my_pred == 0:
            pred = 'neutral'
        else:
            pred = 'negative'
            
        
    return render_template('result.html',prediction = pred)


if __name__ == '__main__':
    
    app.run(port='8088',threaded=True,debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with windowsapi reloader


SystemExit: 1

In [2]:
df = pd.read_csv('all_kindle_review.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'all_kindle_review.csv'

In [14]:
df.head()

,Unnamed: 0,Unnamed: 0.1,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [15]:
df.shape

(12000, 11)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      12000 non-null  int64 
 1   Unnamed: 0.1    12000 non-null  int64 
 2   asin            12000 non-null  object
 3   helpful         12000 non-null  object
 4   rating          12000 non-null  int64 
 5   reviewText      12000 non-null  object
 6   reviewTime      12000 non-null  object
 7   reviewerID      12000 non-null  object
 8   reviewerName    11962 non-null  object
 9   summary         12000 non-null  object
 10  unixReviewTime  12000 non-null  int64 
dtypes: int64(4), object(7)
memory usage: 1.0+ MB


In [17]:
df = df.dropna()
df = df.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1 )

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11962 entries, 0 to 11999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   asin            11962 non-null  object
 1   helpful         11962 non-null  object
 2   rating          11962 non-null  int64 
 3   reviewText      11962 non-null  object
 4   reviewTime      11962 non-null  object
 5   reviewerID      11962 non-null  object
 6   reviewerName    11962 non-null  object
 7   summary         11962 non-null  object
 8   unixReviewTime  11962 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 934.5+ KB


In [21]:
df.rating.value_counts(dropna=False)

 1    5980
-1    3986
 0    1996
Name: rating, dtype: int64

In [19]:
df.loc[(df['rating']==2)|(df['rating']==1),'rating']= -1
df.loc[df['rating']==3,'rating']= 0
df.loc[(df['rating']==4)|(df['rating']==5),'rating']= 1


In [22]:
Sentiment_words=[]

for row in df['rating']:
    
    if row == -1:
        Sentiment_words.append('negative')
    
    elif row == 0:
        Sentiment_words.append('neutral')    
    
    elif row == 1:
        Sentiment_words.append('positive')
    
    else:
        Sentiment_words.append('Failed')
        
df['Sentiment_words'] = Sentiment_words

In [23]:
df.head()

,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,Sentiment_words
0,B0033UV8HI,"[8, 10]",0,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600,neutral
1,B002HJV4DE,"[1, 1]",1,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400,positive
2,B002ZG96I4,"[0, 0]",0,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400,neutral
3,B002QHWOEU,"[1, 3]",0,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400,neutral
4,B001A06VJ8,"[0, 1]",1,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000,positive


In [24]:
def remove_unwant(x):
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
 
    for ele in x:
        if ele in punc:
            x = x.replace(ele, "")
            
    return x        

In [25]:
df['summary'] = df['summary'].apply(lambda x:remove_unwant(x))
df['summary'] = df['summary'].apply(lambda x:x.lower())

In [26]:
df.head()

,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,Sentiment_words
0,B0033UV8HI,"[8, 10]",0,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,entertaining but average,1283385600,neutral
1,B002HJV4DE,"[1, 1]",1,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,terrific menage scenes,1381190400,positive
2,B002ZG96I4,"[0, 0]",0,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,snapdragon alley,1397174400,neutral
3,B002QHWOEU,"[1, 3]",0,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400,neutral
4,B001A06VJ8,"[0, 1]",1,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,book,1356912000,positive


In [ ]:
temp=df['summary'].tolist()
fax=[]

for i in temp:
    fax.append(i)
    
X=pd.Series(fax)

In [27]:
X = df['summary']
y = df['rating']

In [28]:
tf=TfidfVectorizer(lowercase=True)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
clf = MultinomialNB()

In [31]:
X_train = tf.fit_transform(X_train)
X_test = tf.transform(X_test)

In [32]:
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

accuracy_score(y_pred,y_test)

0.6828249059757626

In [33]:
df['summary'][0]

'entertaining but average'

In [38]:
pred = tf.transform(['good'])
clf.predict(pred)

array([1], dtype=int64)